# Tutorial: Recurrent Neural Net in Pytorch

## Part 1: Load MNIST Dataset

In [1]:
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
train_dataset = dsets.MNIST(root='./data/',
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data/',
                           train=False, 
                           transform=transforms.ToTensor())

In [3]:
print(train_dataset.train_data.size())

torch.Size([60000, 28, 28])


In [4]:
print(train_dataset.train_labels.size())

torch.Size([60000])


In [5]:
print(test_dataset.test_data.size())

torch.Size([10000, 28, 28])


In [6]:
print(test_dataset.test_labels.size())

torch.Size([10000])


## Part 2: Make Dataset Iterable

In [7]:
batch_size = 100
n_iters = 3000
# num_epochs = n_iters / (len(train_dataset) / batch_size)
# num_epochs = int(num_epochs)
num_epochs = 2
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

## Part 3: Create RNN Class

In [8]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial states 
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) 
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        
        # Forward propagate RNN
        out, _ = self.lstm(x, (h0, c0))  
        
        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])  
        return out

## Part 4: Instantiate Model Class

In [9]:
# RNN Parameter
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10

In [10]:
rnn = RNN(input_size, hidden_size, num_layers, num_classes)

## Part 5: Instantiate Loss Class

In [11]:
criterion = nn.CrossEntropyLoss()

## Part 6: Instantiate Optimizer Class

In [12]:
learning_rate = 0.01
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)

## Part 7: Train Model

In [13]:
sequence_length = 28

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, sequence_length, input_size))
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = rnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

Epoch [1/2], Step [100/600], Loss: 0.7542
Epoch [1/2], Step [200/600], Loss: 0.2996
Epoch [1/2], Step [300/600], Loss: 0.1765
Epoch [1/2], Step [400/600], Loss: 0.1752
Epoch [1/2], Step [500/600], Loss: 0.2237
Epoch [1/2], Step [600/600], Loss: 0.0832
Epoch [2/2], Step [100/600], Loss: 0.2170
Epoch [2/2], Step [200/600], Loss: 0.0943
Epoch [2/2], Step [300/600], Loss: 0.0765
Epoch [2/2], Step [400/600], Loss: 0.4144
Epoch [2/2], Step [500/600], Loss: 0.1268
Epoch [2/2], Step [600/600], Loss: 0.0944


In [14]:
# Save the Model
torch.save(rnn.state_dict(), 'rnn.pkl')

## Part 8 : Test Model

In [15]:
# Test the Model
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, sequence_length, input_size))
    outputs = rnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total)) 

Test Accuracy of the model on the 10000 test images: 97 %
